In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/grad-cam-eval/grad-cam.csv


In [2]:
!pip install torch torchvision grad-cam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39589 sha256=be1a8c19803a8c8cbc47a8382e259d7a47dca1d62ff0fdfe692d5be4ca923b25
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [3]:
!git clone https://huggingface.co/gotchu/convnext-models
!git clone https://github.com/ashmitg/cxr-image-analysis.git

Cloning into 'label_one'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (13/13), 3.80 KiB | 1.27 MiB/s, done.
Filtering content: 100% (6/6), 7.83 GiB | 127.76 MiB/s, done.
Cloning into 'cxr-image-analysis'...
remote: Enumerating objects: 955, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 955 (delta 8), reused 13 (delta 4), pack-reused 932 (from 1)
Receiving objects: 100% (955/955), 794.97 MiB | 46.91 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (515/515), done.


In [8]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import pytorch_lightning as pl
from torchvision import models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm

evalg_df = pd.read_csv('/kaggle/input/grad-cam-eval/grad-cam.csv')

class CNNModel(pl.LightningModule):
    def __init__(self, num_classes=1000): 
        super(CNNModel, self).__init__()
        self.model = models.convnext_base(pretrained=False, num_classes=num_classes)
    
    def forward(self, x):
        return self.model(x)

# Grad-CAM function
def generate_gradcam(model, image_tensor, class_index):
    def forward_hook(module, input, output):
        model.gradients = output

    target_layer = model.model.features[-1]
    hook = target_layer.register_forward_hook(forward_hook)

    model.zero_grad()
    output = model(image_tensor)
    class_loss = output[0, class_index]
    class_loss.backward()

    gradients = model.gradients
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = model.gradients

    for i in range(activations.size(1)):
        activations[:, i, :, :] *= pooled_gradients[i]

    heatmap = torch.mean(activations, dim=1).squeeze()
    heatmap = np.maximum(heatmap.cpu().detach().numpy(), 0)
    heatmap /= np.max(heatmap)
    
    return heatmap

# Overlay heatmap function
def overlay_heatmap(heatmap, original_image, alpha=0.5, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (original_image.size[0], original_image.size[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)

    original_image = np.array(original_image)
    overlayed_image = cv2.addWeighted(heatmap, alpha, original_image, 1 - alpha, 0)

    return overlayed_image

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image_tensor = transform(image).unsqueeze(0)
    return image, image_tensor

def load_model_from_checkpoint(model_name, model_dir):
    checkpoint_path = os.path.join(model_dir, model_name)
    checkpoint = torch.load(checkpoint_path)
    state_dict = checkpoint['state_dict']

    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith('ema.module.'):
            new_state_dict[k[len('ema.module.'):]] = v
        else:
            new_state_dict[k] = v

    model = CNNModel()
    model.load_state_dict(new_state_dict, strict=False)
    model.eval()
    
    return model

model_dir = '/kaggle/working/label_one/'

output_base_dir = '/kaggle/working/grad-cams/'
os.makedirs(output_base_dir, exist_ok=True)

for idx, row in tqdm(evalg_df.iterrows(), total=len(evalg_df), desc="Processing Grad-CAM"):
    image_path = row['image_path']  # Get image path from DataFrame
    full_image_path = os.path.join('/kaggle/working', image_path)  # Modify image path for Kaggle location
    model_name = row['model_grad_eval']  # Get model name from DataFrame
    view_position = str(row['ViewPosition'])  # Convert ViewPosition to string
    dicom_value = row['dicom']  # Get the dicom value

    model = load_model_from_checkpoint(model_name, model_dir)

    image, image_tensor = load_image(full_image_path)

    with torch.no_grad():
        output = model(image_tensor)
        predicted_class = output.argmax(dim=1)

    heatmap = generate_gradcam(model, image_tensor, predicted_class.item())

    overlayed_image = overlay_heatmap(heatmap, image)

    subject_id = row['subject_id_y']
    subject_folder = f"{subject_id}_x"
    dicom_folder = str(dicom_value)  

    view_position_dir = os.path.join(output_base_dir, view_position, subject_folder, dicom_folder)
    os.makedirs(view_position_dir, exist_ok=True)

    model_prefix = model_name.split('.')[0]  
    output_filename = f"grad_cam_{view_position}_{model_prefix}_{dicom_value}.jpg"
    save_path = os.path.join(view_position_dir, output_filename)

    plt.imsave(save_path, overlayed_image)

    text_file_path = os.path.join(view_position_dir, f"{dicom_folder}.txt")
    with open(text_file_path, 'w') as f:
        for key, value in row.items():
            f.write(f"{key}: {value}\n")  

    print(f"Saved Grad-CAM image at {save_path} and row content at {text_file_path}")


Processing Grad-CAM:   0%|          | 0/214 [00:00<?, ?it/s]/tmp/ipykernel_30/402362521.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoi

Saved Grad-CAM image at /kaggle/working/grad-cams/LATERAL/11812752_x/280b1fc6-0fb16158-0b70bdcf-1b72c58a-1b5daa7d/grad_cam_LATERAL_j_CNN_Label_5_convnext_base_280b1fc6-0fb16158-0b70bdcf-1b72c58a-1b5daa7d.jpg and row content at /kaggle/working/grad-cams/LATERAL/11812752_x/280b1fc6-0fb16158-0b70bdcf-1b72c58a-1b5daa7d/280b1fc6-0fb16158-0b70bdcf-1b72c58a-1b5daa7d.txt


Processing Grad-CAM:   1%|          | 2/214 [00:06<11:52,  3.36s/it]

Saved Grad-CAM image at /kaggle/working/grad-cams/PA/11812752_x/d7de39dc-36561618-9fa3a947-fcb7da6f-7f473aff/grad_cam_PA_j_CNN_Label_5_convnext_base_d7de39dc-36561618-9fa3a947-fcb7da6f-7f473aff.jpg and row content at /kaggle/working/grad-cams/PA/11812752_x/d7de39dc-36561618-9fa3a947-fcb7da6f-7f473aff/d7de39dc-36561618-9fa3a947-fcb7da6f-7f473aff.txt


Processing Grad-CAM:   1%|▏         | 3/214 [00:09<11:34,  3.29s/it]

...

Saved Grad-CAM image at /kaggle/working/grad-cams/AP/17081205_x/bc19420f-5e8eb54b-0b869200-d9cb14d9-189dc1d2/grad_cam_AP_j_CNN_Label_4_convnext_base_bc19420f-5e8eb54b-0b869200-d9cb14d9-189dc1d2.jpg and row content at /kaggle/working/grad-cams/AP/17081205_x/bc19420f-5e8eb54b-0b869200-d9cb14d9-189dc1d2/bc19420f-5e8eb54b-0b869200-d9cb14d9-189dc1d2.txt


Processing Grad-CAM: 100%|█████████▉| 213/214 [11:40<00:03,  3.31s/it]

Saved Grad-CAM image at /kaggle/working/grad-cams/AP/17081205_x/6158aa84-3dbfbe29-5b10b230-456f13b7-bb82200e/grad_cam_AP_j_CNN_Label_1_convnext_base_6158aa84-3dbfbe29-5b10b230-456f13b7-bb82200e.jpg and row content at /kaggle/working/grad-cams/AP/17081205_x/6158aa84-3dbfbe29-5b10b230-456f13b7-bb82200e/6158aa84-3dbfbe29-5b10b230-456f13b7-bb82200e.txt


Processing Grad-CAM: 100%|██████████| 214/214 [11:43<00:00,  3.29s/it]

Saved Grad-CAM image at /kaggle/working/grad-cams/AP/17081205_x/8b2908a7-fdd10c4d-38282f35-fefdac3b-91b8c754/grad_cam_AP_j_CNN_Label_1_convnext_base_8b2908a7-fdd10c4d-38282f35-fefdac3b-91b8c754.jpg and row content at /kaggle/working/grad-cams/AP/17081205_x/8b2908a7-fdd10c4d-38282f35-fefdac3b-91b8c754/8b2908a7-fdd10c4d-38282f35-fefdac3b-91b8c754.txt


In [2]:
import shutil

folder_to_zip = '/kaggle/working/grad-cams'
output_filename = '/kaggle/working/grad-cams.zip'
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)


'/kaggle/working/grad-cams.zip'